## 2.1 多臂老虎机问题 K-armed Bandit Problem

这章是一个引入，介绍了多臂老虎机问题，即在一个有K个臂的老虎机中，每个臂的奖励是一个固定的概率分布，我们的目标是找到一个策略，使得在有限的时间内获得最大的奖励。

在我们的K臂赌博机问题中，$k$ 个动作中的每一个在被选择时都有一个期望或者平均收益，我们称此为这个动作的“__价值__”。我们将在时刻 $t$ 时选择的动作记作 $A_(t)$ ,并将对应的收益记作任一动作 $a$ 对应的价值，记作 $q*(a)$, 是给定动作 $a$ 时收益的期望（动作价值函数）：

$$
q_*(a) \doteq \mathbb{E}[R_t \mid A_t = a]
$$

下面的代码在一个循环中进行 2000 次试验，每次试验中：

随机生成 10 个动作的平均奖励（左右两组）。基于这些平均值生成 2000 个样本数据。

## 2.2 评估动作价值 Action-value Methods

我们使用这些价值的估计来进行动作的选择，这一类方法被统称为 “动作-价值方法”。

$$
 Q_t(a) = \frac{\text{t时刻前通过执行动作} \ a \ \text{得到的收益总和}}{\text{t时刻前执行动作} \ a \ \text{的次数}} = \frac{\sum_{i=1}^{t-1} R_i \cdot \mathbb{1}_{A_i = a}}{\sum_{i=1}^{t-1} \mathbb{1}_{A_i = a}}
$$

指示函数  $\mathbb{1}_{\text{predicate}}$  的值根据条件 (predicate) 的真假来决定其取值，当条件为真时取 1，否则取 0。

最简单的动作选择规则是选择具有最高估计值的动作，即前一节所定义的贪心动作。如果有多个贪心动作，那就任意选择一个，比如随机挑选。我们将这种贪心动作的选择方法记作 ：


- $greedy$ 贪心方法 ： $A_t \doteq \arg\max_{a} Q_t(a)$, 这类方法的一个优点是，如果时刻可以无限长，则每一个动作都会被无限次采样，从而确保所有的 $Qt(a)$ 收敛到 $q*(a)$。

另一种方法是：

- $\epsilon - greedy$ 方法：以较小的概率，从所有动作中随机选择且每个动作的选择概率相等；以较大的概率，选择贪婪动作。选择最优动作的概率会收敛到大于 l-$\epsilon$, 即接近确定性选择。

In [3]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display, HTML

# 记录每次试验的结果
all_means_left = []
all_means_right = []

for _ in range(2000):
    # 随机采样每个动作的平均奖励（左右两组）
    means_left = np.random.normal(size=(10,))
    means_right = np.random.normal(size=(10,))

    # 基于正态分布生成样本数据
    data_left = [np.random.normal(mean, 1.0, 2000) for mean in means_left]
    data_right = [np.random.normal(mean, 1.0, 2000) for mean in means_right]

    all_means_left.append(means_left)
    all_means_right.append(means_right)

# 创建重叠的小提琴图
fig = go.Figure()

for i in range(10):
    # 左侧数据
    fig.add_trace(go.Violin(x=[i]*2000, y=data_left[i],
                            name=f'左 {i+1}',
                            side='negative',
                            line_color='blue',
                            showlegend=False))
    # 右侧数据
    fig.add_trace(go.Violin(x=[i]*2000, y=data_right[i],
                            name=f'右 {i+1}',
                            side='positive',
                            line_color='red',
                            showlegend=False))

# 绘制平均值标记
for i, (mean_left, mean_right) in enumerate(zip(means_left, means_right)):
    fig.add_trace(go.Scatter(x=[i, i], y=[mean_left, mean_right],
                             mode='markers',
                             marker=dict(color=['blue', 'red'], size=8),
                             showlegend=False))
    fig.add_annotation(x=i, y=mean_left, text=f"$q_*^L({i+1})$",
                       showarrow=False, xanchor='right', yanchor='bottom')
    fig.add_annotation(x=i, y=mean_right, text=f"$q_*^R({i+1})$",
                       showarrow=False, xanchor='left', yanchor='bottom')

# 绘制0虚线
fig.add_trace(go.Scatter(x=[-0.5, 9.5], y=[0, 0],
                         mode='lines',
                         line=dict(color='gray', width=1, dash='dash'),
                         showlegend=False))

# 更新布局
fig.update_layout(
    violinmode='overlay',
    xaxis=dict(
        tickmode='array',
        tickvals=np.arange(10),
        ticktext=[f'动作 {i+1}' for i in range(10)],
        title='动作',
        title_font=dict(size=14)
    ),
    yaxis=dict(
        title='奖励分布',
        title_font=dict(size=14)
    ),
    margin=dict(l=50, r=50, t=50, b=50),
    title='重叠的动作奖励分布小提琴图'
)

# 添加图例
fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                         marker=dict(color='blue', size=10),
                         name='左侧数据'))
fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                         marker=dict(color='red', size=10),
                         name='右侧数据'))

# 选项1：在Jupyter环境中显示
display(fig)

# 选项2：保存为HTML文件
pio.write_html(fig, file='./output/overlapping_violin_plot.html', auto_open=True)

print("图表已保存为 './output/overlapping_violin_plot.html'。请在浏览器中打开该文件查看图表。")

图表已保存为 './output/overlapping_violin_plot.html'。请在浏览器中打开该文件查看图表。


## 2.3 10臂测试平台 10-armed Testbed

这节演示了一个测试环境中的概率分布演示。 通过 2000 个随机生成的 k 臂赌博机，$k$ = 10。在每一个赌博机问题中，动作的真实价值为 $q*(a)$, $a$ = 1,...,10, 从一个均值为 0 方差为 1 的标准正态（高斯）分布中选择。当对应于该问题的学习方法在时刻 $t$ 选择 $A_{t}$ 时，实际的收益 $R_{t}$ 则由一个均值为 $q*(A_{t})$ 方差为 1 的正态分布决定。

下面这段模型训练代码（循环1000次）是在基于上面 10 臂测试平台上，比较了上述的贪心方法和两种 $\epsilon$-贪心方法（$\epsilon$ = 0.01 和 $\epsilon$=0.1），并打印出了每种方法的平均奖励和最优动作比例图。

所有方法都用采样平均策略来形成对动作价值的估计。




In [4]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle
from IPython.display import display, HTML

from utils import get_argmax, bandit

SEED = 200
np.random.seed(SEED)


# 运行k臂赌博机算法
def run_bandit(K: int,
               q_star: np.ndarray,
               rewards: np.ndarray,
               optim_acts_ratio: np.ndarray,
               epsilon: float,
               num_steps: int = 1000) -> None:
    Q = np.zeros(K)  # 初始化Q值
    N = np.zeros(K)  # 每个动作被选择的次数    
    ttl_optim_acts = 0

    for i in range(num_steps):
        # 获取动作
        A = None
        if np.random.random() > epsilon:
            A = get_argmax(Q)
        else:
            A = np.random.randint(0, K)

        R, is_optim = bandit(q_star, A)
        N[A] += 1
        Q[A] += (R - Q[A]) / N[A]

        ttl_optim_acts += is_optim
        rewards[i] = R
        optim_acts_ratio[i] = ttl_optim_acts / (i + 1)


# 初始化超参数
K = 10  # 臂的数量
epsilons = [0.0, 0.01, 0.1]
num_steps = 1000
total_rounds = 2000

# 初始化环境
q_star = np.random.normal(loc=0, scale=1.0, size=K)
rewards = np.zeros(shape=(len(epsilons), total_rounds, num_steps))
optim_acts_ratio = np.zeros(shape=(len(epsilons), total_rounds, num_steps))

# 运行k臂赌博机算法
for i, epsilon in enumerate(epsilons):
    for curr_round in range(total_rounds):
        run_bandit(K, q_star,
                   rewards[i, curr_round],
                   optim_acts_ratio[i, curr_round],
                   epsilon,
                   num_steps)

rewards = rewards.mean(axis=1)
optim_acts_ratio = optim_acts_ratio.mean(axis=1)

record = {
    'hyper_params': epsilons,
    'rewards': rewards,
    'optim_acts_ratio': optim_acts_ratio
}

# 创建子图
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("最优动作比率", "平均奖励"),
                    vertical_spacing=0.1)

# 添加最优动作比率的轨迹
for i, epsilon in enumerate(epsilons):
    fig.add_trace(
        go.Scatter(x=list(range(num_steps)), y=optim_acts_ratio[i], name=f'ε={epsilon}'),
        row=1, col=1
    )

# 添加平均奖励的轨迹
for i, epsilon in enumerate(epsilons):
    fig.add_trace(
        go.Scatter(x=list(range(num_steps)), y=rewards[i], name=f'ε={epsilon}'),
        row=2, col=1
    )

# 更新布局
fig.update_layout(height=800, width=800, title_text="K臂赌博机结果")
fig.update_xaxes(title_text="步数", row=1, col=1)
fig.update_xaxes(title_text="步数", row=2, col=1)
fig.update_yaxes(title_text="最优动作比率", row=1, col=1)
fig.update_yaxes(title_text="平均奖励", row=2, col=1)

# 在Jupyter notebook中显示图表
display(fig)

# 保存为HTML
fig.write_html("./output/k_armed_bandit_results.html")

# 将记录保存为pickle文件
with open('./output/2.3-10-armed-test.pkl', 'wb') as f:
    pickle.dump(record, f)

## 2.4 增量式实现 Incremental Implementation

截止目前，我们讨论的动作-价值方法都把动作价值作为观测到的收益的样本均值来估计。下面我们探讨如何才能以一种高效的方式计算这些均值，尤其是如何保持常数级的内存需求和常数级的单时刻计算量。

为了简化标记，我们关心单个动作。令 $R_{i}$ 表示这一动作被选择 $i$ 次后获得的收益，$R{n}$ 表示被选择 $n$ - 1 次后它的估计的动作价值，现在可以简单地把它写为：

$$
Q_{n} = \frac{R_{1} + R_{2} + \cdots + R_{n-1}}{n-1}
$$

上面的这个方法记录所有的奖励，然后在需要估计值时进行计算。但其缺点是，如果这样做，随着更多奖励的出现，内存和计算需求会随着时间增长。每增加一个奖励，就需要额外的内存来存储它，并需要额外的计算来计算分子中的总和。

- 用于更新平均值的增量公式，即 __新估计 ← 旧估计 + 步长 [目标 − 旧估计]__：
$$
\begin{align*}
Q_{n+1} &= \frac{1}{n} \sum_{i=1}^{n} R_i \\
        &= \frac{1}{n} \left( R_n + \sum_{i=1}^{n-1} R_i \right) \\
        &= \frac{1}{n} \left( R_n + (n-1) \frac{1}{n-1} \sum_{i=1}^{n-1} R_i \right) \\
        &= \frac{1}{n} \left( R_n + (n-1) Q_n \right) \\
        &= \frac{1}{n} \left( R_n + n Q_n - Q_n \right) \\
        &= Q_n + \frac{1}{n} \left[ R_n - Q_n \right]
\end{align*}
$$
- 综合起来：简单的多臂老虎机算法伪代码：
$$
\begin{aligned}
&\textbf{初始化, 令 } a = 1 \text{ 到 } k: \\
&\quad Q(a) \leftarrow 0 \\
&\quad N(a) \leftarrow 0 \\
&\textbf{无限循环：} \\
&\quad A \leftarrow 
\begin{cases} 
    \arg\max_a Q(a) & \text{ 以 } 1 - \epsilon \text{ 概率 (随机跳出贪心)} \\
    \text{一个随机的动作 } & \text{以 } \epsilon \text{ 概率 }
\end{cases} \\
&\quad R \leftarrow \text{bandit}(A) \\
&\quad N(A) \leftarrow N(A) + 1 \\
&\quad Q(A) \leftarrow Q(A) + \frac{1}{N(A)} [R - Q(A)]
\end{aligned}
$$



## 2.5 跟踪一个非平稳问题 Tracking a Nonstationary Problem

到目前为止我们讨论的取平均方法对平稳的赌博机问题是合适的，即收益的概率分布不随着时间变化的赌博机问题但如果赌博机的收益概率是随着时间变化的，该方法就不合适。

针对到非平稳的强化学习问题，给 __近期的收益__ 赋予比过去很久的收益更高的权值就是一种合理的处理方式。

最流行的方法之一是使用固定步长。比如说，用于更新 $n$ - 1 个过去的收益的均值 $Q{n}$ 的增量更新规则：

$$
Q_{n+1} \doteq Q_n + \alpha \left[ R_n - Q_n \right]
$$

其中，步长参数 $\alpha \in (0, 1]$ 是一个常数。即使 $Q_{n+1}$ 成为对过去的收益和初始的估计 $Q{i}$ 的加权平均。

证明过程：

$$
\begin{align*}
Q_{n+1} &= Q_n + \alpha \left[ R_n - Q_n \right] \\
        &= \alpha R_n + (1 - \alpha) Q_n \\
        &= \alpha R_n + (1 - \alpha) \left[ \alpha R_{n-1} + (1 - \alpha) Q_{n-1} \right] \\
        &= \alpha R_n + (1 - \alpha) \alpha R_{n-1} + (1 - \alpha)^2 Q_{n-1} \\
        &= \alpha R_n + (1 - \alpha) \alpha R_{n-1} + (1 - \alpha)^2 \alpha R_{n-2} + \cdots + (1 - \alpha)^{n-1} \alpha R_1 + (1 - \alpha)^n Q_1 \\
        &= (1 - \alpha)^n Q_1 + \sum_{i=1}^{n} \alpha (1 - \alpha)^{n-i} R_i
\end{align*}
$$

因为 $1 - \alpha$ 小于 1，因此随着间隔奖励数量的增加，分配给 $R_i$ 的权重会减少。这个方法有时候也被称为“指数近因加权平均”。

## 2.6 乐观初始值 Optimistic Initial Values

目前为止我们讨论的所有方法都在一定程度上依赖于初始动作值 $Q_{1}(a)$ 的选择。从统计学角度来说，这些方法 (由于初始估计值) 是有偏置的。

初始动作的价值同时也提供了一种简单的试探方式。比如一个 10 臂的测试平台，我们替换掉原先的初始值 0,将它们全部设为 +5。

In [5]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle
from IPython.display import display, HTML

from utils import get_argmax, bandit

SEED = 200
np.random.seed(SEED)

# 运行k臂赌博机算法
def run_bandit(K:int,
               q_star:np.ndarray,
               rewards:np.ndarray,
               optim_acts_ratio:np.ndarray,
               epsilon:float,
               num_steps:int=1000,
               init_val:int=0) -> None:

    Q = np.ones(K) * init_val # 使用OIV初始化Q值
    ttl_optim_acts = 0
    alpha = 0.1

    for i in range(num_steps):
        # 获取动作
        A = None
        if np.random.random() > epsilon:
            A = get_argmax(Q)
        else:
            A = np.random.randint(0, K)

        R, is_optim = bandit(q_star, A)
        Q[A] += alpha * (R - Q[A])

        ttl_optim_acts += is_optim
        rewards[i] = R
        optim_acts_ratio[i] = ttl_optim_acts / (i + 1)

# 初始化超参数
K = 10 # 臂的数量
epsilons = [0.1, 0.0]
init_vals = [0.0, 5.0]
num_steps = 1000
total_rounds = 2000

# 初始化环境
q_star = np.random.normal(loc=0, scale=1.0, size=K)
rewards = np.zeros(shape=(len(epsilons), total_rounds, num_steps))
optim_acts_ratio = np.zeros(shape=(len(epsilons), total_rounds, num_steps))

# 运行k臂赌博机算法
for i, (epsilon, init_val) in enumerate(zip(epsilons, init_vals)):
    for curr_round in range(total_rounds):
        run_bandit(K, q_star,
                   rewards[i, curr_round],
                   optim_acts_ratio[i, curr_round],
                   epsilon=epsilon,
                   num_steps=num_steps,
                   init_val=init_val)

rewards = rewards.mean(axis=1)
optim_acts_ratio = optim_acts_ratio.mean(axis=1)

record = {
    'hyper_params': [epsilons, init_vals],
    'rewards': rewards,
    'optim_acts_ratio': optim_acts_ratio
}

# 创建子图
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("平均奖励", "最优动作比率"),
                    vertical_spacing=0.1)

# 添加平均奖励的轨迹
for i, (epsilon, init_val) in enumerate(zip(epsilons, init_vals)):
    fig.add_trace(
        go.Scatter(x=list(range(num_steps)), y=rewards[i],
                   name=f'ε={epsilon}, init_val={init_val}'),
        row=1, col=1
    )

# 添加最优动作比率的轨迹
for i, (epsilon, init_val) in enumerate(zip(epsilons, init_vals)):
    fig.add_trace(
        go.Scatter(x=list(range(num_steps)), y=optim_acts_ratio[i],
                   name=f'ε={epsilon}, init_val={init_val}'),
        row=2, col=1
    )

# 更新布局
fig.update_layout(height=800, width=800, title_text="K臂赌博机结果：不同ε和初始值的比较")
fig.update_xaxes(title_text="步数", row=1, col=1)
fig.update_xaxes(title_text="步数", row=2, col=1)
fig.update_yaxes(title_text="平均奖励", row=1, col=1)
fig.update_yaxes(title_text="最优动作比率", row=2, col=1)

# 在Jupyter notebook中显示图表
display(fig)

# 保存为HTML
fig.write_html("./output/k_armed_bandit_results_oiv.html")

# 将记录保存为pickle文件
with open('./output/OIV_record.pkl', 'wb') as f:
    pickle.dump(record, f)

## 2.7 置信度上界动作选择 Upper-Confidence-Bound Action Selection

因为对 __动作-价值__ 的估计总会存在不确定性，所以试探是必须的。
虽然贪心动作在当前时刻看起来最好，但实际上其他一些动作可能从长远看更好。
$\epsilon$―贪心算法会尝试选择非贪心的动作，但是这是一种盲目的选择，因为它不大会去选择接近贪心或者不确定性特别大的动作。

在非贪心动作中，最好是根据它们的潜力来选择可能事实上是最优的动作，这就要 __考虑到它们的估计有多接近最大值，以及这些估计的不确定性。__

$$
\begin{flalign*}
A_t = \arg\max_{a} \left[ Q_t(a) + c \sqrt{ \frac{\text{ln}t}{N_{t}(a)}}  \right],
\end{flalign*}
$$

这个公式中，$\text{ln }t$ 表示 $t$ 的自然对数，$N_{t}(a)$ 表示在 $t$ 步之前动作 $a$ 被选择的次数。$c$ 是一个大于0的超参数用于控制“试探”的程度（即置信）。

- 平方根的作用是对动作 $a$ 值估计的不确定性或方差。
- c 决定置信度
- 当每次 $a$ 动作被选择时，它的不确定性降低。
- 当每次 $a$ 动作 __没有__ 被选择时，由于分子中的 $t$ 增大但分母 $N_{t}(a)$ 没有变化，不确定性增加。

所以最终效果是：那些较低价值估计的动作、或者已经被选择更多的动作最终被选择到的机率较低。

下面这段代码通过2000个步骤循环模拟了比较ε-贪心算法和上置信界(UCB)算法在K臂赌博机问题上的表现（运行需约1分钟），帮助我们理解这两种算法的优缺点和适用场景。通过交互式图表，我们可以更清晰地观察到算法性能随时间的变化趋势：

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle
from IPython.display import display, HTML

from utils import get_argmax, bandit

SEED = 200
np.random.seed(SEED)

# 运行k臂赌博机算法
def run_bandit(K:int,
               q_star:np.ndarray,
               rewards:np.ndarray,
               optim_acts_ratio:np.ndarray,
               epsilon: float,
               num_steps:int=1000) -> None:

    Q = np.zeros(K)
    N = np.zeros(K) # 每个动作被选择的次数    
    ttl_optim_acts = 0

    for i in range(num_steps):
        A = None
        # 获取动作
        if np.random.random() > epsilon:
            A = get_argmax(Q)
        else:
            A = np.random.randint(0, K)

        R, is_optim = bandit(q_star, A)
        N[A] += 1
        Q[A] += (R - Q[A]) / N[A]

        ttl_optim_acts += is_optim
        rewards[i] = R
        optim_acts_ratio[i] = ttl_optim_acts / (i + 1)


# 运行带UCB的赌博机算法
def run_bandit_UCB(K:int,
                   q_star:np.ndarray,
                   rewards:np.ndarray,
                   optim_acts_ratio: np.ndarray,
                   c: float,
                   num_steps:int=1000) -> None:

    Q = np.zeros(K)
    N = np.zeros(K) # 每个动作被选择的次数    
    ttl_optim_acts = 0

    for i in range(num_steps):
        A = None

        # 避免除以0：
        # 如果N中有0，则选择N=0的动作
        if (0 in N):
            candidates = np.argwhere(N == 0).flatten()
            A = np.random.choice(candidates)
        else:
            confidence = c * np.sqrt(np.log(i) / N)
            freqs = Q + confidence
            A = np.argmax(freqs).flatten()

        R, is_optim = bandit(q_star, A)
        N[A] += 1
        Q[A] += (R - Q[A]) / N[A]

        ttl_optim_acts += is_optim
        rewards[i] = R
        optim_acts_ratio[i] = ttl_optim_acts / (i + 1)



# 初始化超参数
K = 10 # 臂的数量
num_steps = 1000
total_rounds = 2000
q_star = np.random.normal(loc=0, scale=1.0, size=K)

hyper_params = {'UCB': 2, 'epsilon': 0.1}

rewards = np.zeros(shape=(len(hyper_params), total_rounds, num_steps))
optim_acts_ratio = np.zeros(shape=(len(hyper_params), total_rounds, num_steps))

# 运行ε-贪心赌博机算法
for curr_round in range(total_rounds):
    run_bandit(K,
               q_star,
               rewards[0, curr_round],
               optim_acts_ratio[0, curr_round],
               epsilon=hyper_params['epsilon'],
               num_steps=num_steps)

# 运行UCB算法并获取记录
for curr_round in range(total_rounds):
    run_bandit_UCB(K,
                   q_star,
                   rewards[1, curr_round],
                   optim_acts_ratio[1, curr_round],
                   c=hyper_params['UCB'],
                   num_steps=num_steps)

rewards = rewards.mean(axis=1)
optim_acts_ratio = optim_acts_ratio.mean(axis=1)

record = {
    'hyper_params': hyper_params,
    'rewards': rewards,
    'optim_acts_ratio': optim_acts_ratio
}

# 创建子图
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("最优动作比率", "平均奖励"),
                    vertical_spacing=0.1)

# 添加最优动作比率的轨迹
fig.add_trace(
    go.Scatter(x=list(range(num_steps)), y=optim_acts_ratio[0], name='ε-贪心'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=list(range(num_steps)), y=optim_acts_ratio[1], name='UCB'),
    row=1, col=1
)

# 添加平均奖励的轨迹
fig.add_trace(
    go.Scatter(x=list(range(num_steps)), y=rewards[0], name='ε-贪心'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=list(range(num_steps)), y=rewards[1], name='UCB'),
    row=2, col=1
)

# 更新布局
fig.update_layout(height=800, width=800, title_text="K臂赌博机结果：ε-贪心 vs UCB")
fig.update_xaxes(title_text="步数", row=1, col=1)
fig.update_xaxes(title_text="步数", row=2, col=1)
fig.update_yaxes(title_text="最优动作比率", row=1, col=1)
fig.update_yaxes(title_text="平均奖励", row=2, col=1)

# 在Jupyter notebook中显示图表
display(fig)

# 保存为HTML
fig.write_html("./output/k_armed_bandit_results_ucb.html")

# 将记录保存为pickle文件
with open('./output/UCB_record.pkl', 'wb') as f:
    pickle.dump(record, f)

## 2.8 梯度赌博机 Bandit with Gradient

以上的几种方法都是可选方法，但并不是唯一可使用的方法。

2.8这节所讲的梯度方法就是针对每一个动作 $a$ 学习一个它的偏好函数 $H_{t}(a)$ 。偏好函数值越大，它动作就越频繁地被选择。

$$
\text{Pr}\{A_t = a\} \doteq \frac{e^{H_t(a)}}{\sum_{b=1}^{k} e^{H_t(b)}} \doteq \pi_t(a)
$$

其中 $\pi_t(a)$ 用来表示动作 a 在时刻 t 时被选择的概率。

## 2.9 关联搜索（上下文相关的赌博机） Associative Search (Contextual Bandit)

这篇没有什么笔记。

## 2.10 总结 Summary

这篇是对整个第二章的总结，没有什么笔记。
